In [8]:
# %load notebook_setup.py
#Use %load notebook_setup.py to load these commands at the top of your
#ipython notebook.  Includes running the std_libraries.py file. 

%run std_libraries.py
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook

what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = othera
base directory is C:\Users\BMH_work\github\yeast_esr_expression_analysis
Importing yeast_esr_exp and setting base_dir and data_processing_dir
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
#Make different gene sets to look up orthologs (using SC_genenames as a key).   For SC will have all the genes, for KL will only have
#genes with SC orthologs. 

gene_sets, background_map, background_genes = yeast_esr_exp.build_pka_act_rep_target_sets()

gene_set_names = ['klsc_act', 'klsc_rep','sc_only_act', 'sc_only_rep','kl_only_act', 'kl_only_rep']

#Set up go term list and database: 
GO_aspect = 'P'
#The three GO_aspect are: 
#C = cellular_component
#F = molecular_function
#P = biological_process
go_slims_aspect, go_term_list = yeast_esr_exp.load_goslim_data(GO_aspect, go_slim_fname = 'go_slim_mapping_20181204.tab')

go_term_enrichment = {}
pval = 0.1
FDR = 0.05

for gene_set_name in gene_set_names: 
    gene_set = gene_sets[gene_set_name] 
    print(gene_set_name)
    go_term_enrichment_raw = yeast_esr_exp.go_term_enrichment(gene_set, background_genes[background_map[gene_set_name]], go_term_list, go_slims_aspect)
    (result,padj,_,_) = sm.stats.multipletests(go_term_enrichment_raw['pvalue'],alpha=FDR, method = 'fdr_bh')
    go_term_enrichment_raw['detect_FDR']=result
    go_term_enrichment_raw['padj'] = padj
    go_term_enrichment_filtered = go_term_enrichment_raw[go_term_enrichment_raw['pvalue']<pval].sort_values('pvalue')
    go_term_enrichment[gene_set_name] = go_term_enrichment_filtered

#fname = data_processing_dir + "go_terms/go_enrichment_20181204.xls"
fname = data_processing_dir + "go_terms/go_enrichment_20200609.xls"
writer = pd.ExcelWriter(fname)
for gene_set_name, go_term_enrichment_set in go_term_enrichment.items(): 
    go_term_enrichment_set.to_excel(writer, sheet_name = gene_set_name)
writer.save()

#This sets a longer column width if you want to try to look at genes here rather than in the file. 
#pd.set_option('display.max_colwidth', -1)

EBG00000981603 from pkainh_act not in kl ortholog index
EBG00000982260 from pkainh_act not in kl ortholog index
EBG00000982177 from pkainh_act not in kl ortholog index
EBG00000981954 from pkainh_act not in kl ortholog index
EBG00000982146 from pkainh_act not in kl ortholog index
EBG00000982140 from pkainh_act not in kl ortholog index
EBG00000982282 from pkainh_act not in kl ortholog index
EBG00000981933 from pkainh_act not in kl ortholog index
KLLA0A06072r from pkainh_act not in kl ortholog index
KLLA0C17908r from pkainh_act not in kl ortholog index
KLLA0C17886r from pkainh_act not in kl ortholog index
EBG00000981852 from pkainh_act not in kl ortholog index
EBG00000981576 from pkainh_act not in kl ortholog index
EBG00000982237 from pkainh_act not in kl ortholog index
EBG00000981753 from pkainh_act not in kl ortholog index
EBG00000982313 from pkainh_act not in kl ortholog index
EBG00000982298 from pkainh_act not in kl ortholog index
KLLA0B10494r from pkainh_act not in kl ortholog index


Table [go_enrichment]: Go term enrichment was calculated from the Go Slim Dataset [ref?] downloaded from SGD on 20180412 and p-values were calculated using fisher's exact test against a background of either all genes in S.Cerevisiae (sc_only groups) or all genes in K.Lactis that contain an SC ortholog (kl_only groups and klsc groups)

In [60]:
#various gene sets
#genes = go_term_df_cutoff.loc["lipid metabolic process"]['genes common name']
#genes = ['XBP1','RME1','COX1','COX4','COX13','COX6','GSM1','COX12','CYT1']
#genes =  ['BAS1', 'MET32', 'GZF3','GLN3', 'MET28', 'MET4', 'GAT1'] #Purine Metabolism
#genes = ['ADE1','ADE2','ADE4','ADE5','ADE6','ADE7','ADE8','ADE12','ADE13','ADE16']  #Purine Genes
#genes = ['ZWF1','HIS4','SFP1']


In [50]:
# terms_of_interest = ["lipid metabolic process","biological_process","conjugation","response to chemical",
#                      "sporulation","endosomal transport","lipid transport","response to oxidative stress",
#                      "cellular respiration","cofactor metabolic process","carbohydrate transport",
#                      "mitochondrial translation", "mitochondrion organization","translational initiation",
#                      "cellular amino acid metabolic process", "cell budding",
#                      "nucleobase containing small molecule metabolic process", "snoRNA processing", "amino acid transport"]

# term_df_dict = {}
# for term in terms_of_interest:
#     term_genes = list(go_slims_aspect[go_slims_aspect['GO_term']==term]['sc_genename'])
#     kl_sc_PKA_data_term = kl_sc_PKA_data[kl_sc_PKA_data['sc_genename'].isin(term_genes)]
#     term_df_dict[term]=kl_sc_PKA_data_term

# kl_sc_sig_unsig_dict = {'Unsig KL and SC' : kl_sc_PKA_data_unsig, 
#                  'Sig KL only': kl_sc_PKA_data_klsig_scunsig, 
#                  'Sig SC only': kl_sc_PKA_data_scsig_klunsig, 
#                  'Sig KL and SC': kl_sc_PKA_data_klscsig}

# condition_set = {}
# for df_dict in (term_df_dict,kl_sc_sig_unsig_dict): 
#     condition_set.update(df_dict)



In [61]:
# #Plot a particular set of go terms on top of my scatter plot

# #plot figure in plot.ly
# # condition_set = {'Unsig KL and SC' : kl_sc_PKA_data_unsig, 
# #                  'Sig KL only': kl_sc_PKA_data_klsig_scunsig, 
# #                  'Sig SC only': kl_sc_PKA_data_scsig_klunsig, 
# #                  'Sig KL and SC': kl_sc_PKA_data_klscsig,
# #                  term: kl_sc_PKA_data_term }

# fig_data = [
#   		{
#   			'x': condition_set[condition]['PKA(AS)+1NMPP1_KL'], 
#         	'y': condition_set[condition]['PKA(AS)+1NMPP1_SC'], 
#         	'text': condition_set[condition]['SC_common_name'], 
#         	'mode': 'markers', 
#         	'name': condition} for condition in condition_set.keys()]
# fig = {
#     'data': fig_data,
#     'layout': {
#         'xaxis': {'title': 'KL LFC 4uM NMPP1'},
#         'yaxis': {'title': 'SC LFC 4uM NMPP1'}
#     }
# }

# # IPython notebook
# # py.iplot(fig, filename='pandas/multiple-scatter')

# py.iplot(fig, filename='sc_kl_scatter_sig_highlight_20170915_various_goterms')


# #use inline plotly. 

In [213]:
#Generate a list of all genes in KL_SC_PKA_data and add as a list in intermine named
#20170905_kl_sc_PKA_data_genes 
#We can then make subsets programmatically using the API

#I ran this command: 
#" ".join(kl_sc_PKA_data['sc_genename'])

#Then copied and pasted the output into the yeastmine build query list. 

#The following items were not found in SGD: 
#Scer_YGOB_YDR134C Scer_RDN18-1 Scer_YGOB_SDC25 Scer_RDN25-1 Scer_RDN58-1 Scer_YGOB_Anc_7.495 Scer_YGOB_ADL119W Scer_RDN5-1

#I wonder if I could do this just using something like this: query.add_constraint("Gene", "LOOKUP", ["YNL323W","YIL093C",...])



In [62]:
# mitophagy_list = ['YCR079W','YDL113C','YFR021W','YGR049W','YIL146C','YJL036W','YLL042C','YLR423C','YNL223W','YNR007C']
# mito_disassembly_list = ['YCR079W','YDL113C','YFR021W','YGR049W','YIL146C','YJL036W','YLL042C','YLR423C','YNL223W','YNR007C']
# lipid_catabolic_list = ['YAL051W','YCR068W','YDR036C','YDR058C','YGL010W','YKR009C','YKR089C','YMR008C','YMR313C','YOL011W','YOR059C','YOR081C','YOR084W','YOR363C','YPR001W','YPR002W','YPR006C']
# #kl_sc_PKA_data[kl_sc_PKA_data['sc_genename'].isin(lipid_catabolic_list)]

In [63]:
# #Generate Query for all kl_sc_PKA_data genes

# # Get a new query on the class (table) you will be querying:
# query_all = service.new_query("Gene")

# # The view specifies the output columns
# query_all.add_view(
#     "primaryIdentifier", "secondaryIdentifier", "organism.shortName", "symbol",
#     "name"
# )

# # You can edit the constraint values below
# query_all.add_constraint("Gene", "IN", "20170509_kl_sc_PKA_data_genes", code = "A")

# # Uncomment and edit the code below to specify your own custom logic:
# # query.set_logic("A")

# #gene_id_list = [str(row["id"] for row in query.rows()]

# #type(gene_id_list[0])

# # query_list = list(query)

# # for row in query_list:     #query.rows():
# #     print(row["primaryIdentifier"], row["secondaryIdentifier"], row["organism.shortName"], \
# #         row["symbol"], row["name"], row["id"])

In [64]:
# # Yeastmine queries for each geneset
# #For each set of genes
# # gene_sets = [('klsig','act'),
# #             ('scsig','act'),
# #             ('klscsig','act'),
# #             ('klsig','rep'),
# #             ('scsig','rep'),
# #             ('klscsig','rep')]

# #Add in the one gene that is significant for both but repressed in SC and activated in KL. 
# klsig_act_genes = list(kl_sc_PKA_data_klsig_scunsig[kl_sc_PKA_data_klsig_scunsig['PKA(AS)+1NMPP1_KL']>0]['sc_genename'])
# klsig_act_genes.append('YJL208C')

# #Add in the one gene that is significant for both but activated in SC and repressed in KL.
# klsig_rep_genes = list(kl_sc_PKA_data_klsig_scunsig[kl_sc_PKA_data_klsig_scunsig['PKA(AS)+1NMPP1_KL']<0]['sc_genename'])
# klsig_rep_genes.append('YNL014W')

# #Add in the one gene that is significant for both but activated in SC and repressed in KL.
# scsig_act_genes = list(kl_sc_PKA_data_scsig_klunsig[kl_sc_PKA_data_scsig_klunsig['PKA(AS)+1NMPP1_SC']>0]['sc_genename'])
# scsig_act_genes.append('YNL014W')

# #Add in the one gene that is significant for both but repressed in SC and activated in KL. 
# scsig_rep_genes = list(kl_sc_PKA_data_scsig_klunsig[kl_sc_PKA_data_scsig_klunsig['PKA(AS)+1NMPP1_SC']<0]['sc_genename'])
# scsig_rep_genes.append('YJL208C')

# gene_sets = ['klsig_act', 'klsig_rep','scsig_act','scsig_rep','klscsig_act','klscsig_rep']

# gene_set_dict = {'klsig_act': klsig_act_genes,
#                  'klsig_rep': klsig_rep_genes,
#                  'scsig_act': scsig_act_genes,
#                  'scsig_rep': scsig_rep_genes,
#                  'klscsig_act': list(kl_sc_PKA_data_klscsig[(kl_sc_PKA_data_klscsig['PKA(AS)+1NMPP1_SC']>0) & (kl_sc_PKA_data_klscsig['PKA(AS)+1NMPP1_KL']>0)]['sc_genename']),
#                  'klscsig_rep': list(kl_sc_PKA_data_klscsig[(kl_sc_PKA_data_klscsig['PKA(AS)+1NMPP1_SC']<0) & (kl_sc_PKA_data_klscsig['PKA(AS)+1NMPP1_KL']<0)]['sc_genename'])
#                 }

# missing_genes = ['Scer_YGOB_YDR134C', 'Scer_RDN18-1', 
#                  'Scer_YGOB_SDC25', 'Scer_RDN25-1', 
#                  'Scer_RDN58-1', 'Scer_YGOB_Anc_7.495', 
#                  'Scer_YGOB_ADL119W', 'Scer_RDN5-1']

# pval_cutoff = 0.01

# for gene_set in gene_sets: 
#     print(gene_set)
#     gene_set_list = gene_set_dict[gene_set]
    
#     #filter out missing genes 
#     print("The following missing genes are filtered out:")
#     print(set(gene_set_list) & set(missing_genes))
#     gene_set_list_filtered = list(set(gene_set_list)-set(missing_genes))
#     new_query = query_all.clone()
    
#     #subset query based on desired gene set
#     new_query.add_constraint("Gene.secondaryIdentifier", "ONE OF", gene_set_list_filtered)
    
#     #Generate a list
#     new_list = service.create_list(new_query.select('Gene.id'), "Gene", name = "20170905" + gene_set)
    
#     #Run Go enrichment on the list
#     results = new_list.calculate_enrichment('go_enrichment_for_gene')
#     results_list = list(results)    
#     for result in results_list: 
#         print(result)
#         if result['p-value']<pval_cutoff:
#             print(result)
#             print(result)
            
#     #Given a P-value cutoff
#     # Save the go term 
#     # Save the contingency table
#     # Save the related genes

# ##To see a list of widgets that are available
# service.widgets 

#results = new_list.calculate_enrichment('go_enrichment_for_gene')


In [ ]:
# #Get genes from a go term.  

# from intermine.webservice import Service
# service = Service("http://yeastmine.yeastgenome.org:443/yeastmine/service")
# query = service.new_query("Gene")
# query.add_view(
#     "primaryIdentifier", "secondaryIdentifier", "symbol", "featureType",
#     "goAnnotation.qualifier", "goAnnotation.ontologyTerm.identifier",
#     "goAnnotation.ontologyTerm.name", "goAnnotation.ontologyTerm.namespace",
#     "goAnnotation.evidence.code.code", "goAnnotation.evidence.code.withText",
#     "goAnnotation.evidence.code.annotType",
#     "goAnnotation.evidence.publications.pubMedId",
#     "goAnnotation.evidence.publications.citation"
# )
# query.add_sort_order("Gene.secondaryIdentifier", "ASC")
# query.add_constraint("goAnnotation.ontologyTerm", "LOOKUP", "GO:0000182", code = "A")

# for row in query.rows():
#     print row["primaryIdentifier"], row["secondaryIdentifier"], row["symbol"], row["featureType"], \
#         row["goAnnotation.qualifier"], row["goAnnotation.ontologyTerm.identifier"], \
#         row["goAnnotation.ontologyTerm.name"], row["goAnnotation.ontologyTerm.namespace"], \
#         row["goAnnotation.evidence.code.code"], row["goAnnotation.evidence.code.withText"], \
#         row["goAnnotation.evidence.code.annotType"], \
#         row["goAnnotation.evidence.publications.pubMedId"], \
#         row["goAnnotation.evidence.publications.citation"]

In [7]:
# #See which genes correspond to JSO targets. 
# jso_msn2_targets = ['SSA1', 'SSA4', 'HSP104', 'CTT1', 'TPK1', 'TFS1', 'DDR2', 'TSA1', 'HSP12', 'TMA10', 'TPS2', 'RTC3', 'GPH1', 'HXK1', 'TSA2', 'CYC7', 'PNC1', 'GSY1', 'FMP48', 'SS13', 'ALD4', 'YNR014W', 'HOR2', 'YOR173W', 'RAS2', 'GLK1', 'GPD1', 'HSP26', 'SIP18', 'TKL2', 'FMP16', 'ALD3', 'PGM2']


In [65]:
# gene_set_data = kl_sc_PKA_data[kl_sc_PKA_data['SC_common_name'].isin(jso_msn2_targets)]
# gene_set_list = gene_set_data['sc_genename']


# go_term_data = []

# go_term_index = []

# for term in go_term_list: 
#     term_genes = list(go_slims_aspect[go_slims_aspect['GO_term']==term]['sc_genename'])
#     if len(term_genes)> len(set(term_genes)):
#         print("Duplicate Term: " + term)
#     subset_genes_in_goterm =  set(gene_set_list) & set(term_genes)
#     N_subset_genes_in_goterm = len(subset_genes_in_goterm)
#     N_genes_in_goterm = len(term_genes)
#     if N_subset_genes_in_goterm >0:
#         subset_genes_in_goterm_commonname = io_library.SC_common_name_lookup(subset_genes_in_goterm)
#         go_term_data.append((N_subset_genes_in_goterm,
#                              subset_genes_in_goterm,
#                              subset_genes_in_goterm_commonname,
#                              N_genes_in_goterm))
#         go_term_index.append(term)

# go_term_df = pd.DataFrame(go_term_data, index = go_term_index,columns = ['N subset genes in goterm',
#                                                                             'genes',
#                                                                             'genes common name',
#                                                                             'N genes in goterm'])
# go_term_df[['genes common name','N genes in goterm']]